In [2]:
import pandas as pd

# 讀取 CSV 檔案到 DataFrame
file_path = "data/eia_crude_oil_imports_revised.csv"
df = pd.read_csv(file_path)

# 檢視 DataFrame
print(df.head())  # 顯示前五筆資料

   period originName  quantity    quantity-units
0  20-Jan    Ecuador       730  thousand barrels
1  20-Jan     Canada        87  thousand barrels
2  20-Jan     Canada        47  thousand barrels
3  20-Jan     Canada       344  thousand barrels
4  20-Jan     Canada       467  thousand barrels


針對出口地，將資料整理成每月的各原油出口量

In [2]:
Brent = ["Cameroon", "Angola", "United Kingdom", "Norway", "Nigeria", "Libya", "Ghana", "Italy", "Congo-Brazzaville", 
         "Algeria", "South Sudan", "Netherlands", "Gabon", "Tunisia", "Equatorial Guinea", "Cote d'Ivoire", "Chad" ]
Wti = ["Ecuador", "Canada", "Colombia", "Brazil", "Argentina", "Trinidad and Tobago", "Peru", "Mexico", "Guyana", 
       "Guatemala", "Panama", "Venezuela"]
Dubai = ["Brunei", "Saudi Arabia", "Kuwait", "Iraq", "Egypt", "Vietnam", "United Arab Emirates", "Indonesia", "Qatar"]
other = ["Russia", "Kazakhstan", "The Bahamas", "Country not known", "Australia"]

def classify_origin(origin):
    if origin in Brent:
        return "Brent"
    elif origin in Wti:
        return "WTI"
    elif origin in Dubai:
        return "Dubai"
    else:
        return "Unknown"
    
def parse_date(date):
    # 分離年份和月份
    year = int(date[:2]) + 2000  # 將 "20" 轉為 2020，"24" 轉為 2024
    month = date[3:]  # 提取月份
    return f"1-{month}-{year}"  # 返回標準日期格式，例如 "1-Jan-2020"

    

df['period'] = df['period'].apply(parse_date)
df['period'] = pd.to_datetime(df['period'], format='%d-%b-%Y')
df['period'] = df['period'].dt.to_period('M').astype(str)

df['category'] = df['originName'].apply(classify_origin)

# 創建一個新的 DataFrame，將數據按照基準重構
pivot_df = df.pivot_table(
    index='period',  # 使用 period 作為行索引
    columns='category',  # 使用 category 作為列
    values='quantity',  # 對 quantity 進行聚合
    aggfunc='sum',  # 聚合方法為求和
    fill_value=0  # 如果沒有數據，填充為 0
).reset_index()

# 將結果保存為新的 CSV（可選）
pivot_df.to_csv("monthly_export_data.csv", index=False, encoding='utf-8')

# 打印結果
print(pivot_df)

category   period   Brent  Dubai  Unknown      WTI
0         2020-01   45402  28042   175105  1142575
1         2020-02   39431  23156   159859  1100967
2         2020-03   20594  41608   176848  1127217
3         2020-04   31255   9933   116340  1006635
4         2020-05   54600    917   302148   963186
5         2020-06   59143  15778   297227   970375
6         2020-07   16562  18879   187061  1059107
7         2020-08   39704  22531   102084   990005
8         2020-09   40495  18004    94269   980714
9         2020-10   51835  31507   104909   963207
10        2020-11   52976  22232    85519  1019858
11        2020-12   71386  13503    43463  1121827
12        2021-01   76482  25228    71295  1096102
13        2021-02   80143  10885    78792   931847
14        2021-03   73141  63919    96270   940619
15        2021-04   87374  43526   114310   977410
16        2021-05   78253  64904    96866  1033501
17        2021-06  105511  64253   129332  1097985
18        2021-07  103054  6620

針對出口量進行週期性加權平滑處理(將月資料調整成日資料)

In [6]:
import numpy as np
# 讀取 CSV 檔案到 DataFrame
file_path = "data/monthly_export_data.csv"
df = pd.read_csv(file_path)

# 檢視 DataFrame
print(df.head())  # 顯示前五筆資料

    period  Brent  Dubai  Unknown      WTI
0  2020-01  45402  28042   175105  1142575
1  2020-02  39431  23156   159859  1100967
2  2020-03  20594  41608   176848  1127217
3  2020-04  31255   9933   116340  1006635
4  2020-05  54600    917   302148   963186


In [ ]:
def expand_to_daily(row, column):
    # 獲取月份和數據總量
    period = pd.to_datetime(row['period'])
    total_quantity = row[column]
    
    # 獲取該月份的所有日期
    days_in_month = pd.date_range(start=period, periods=period.days_in_month, freq='D')
    
    # 設置每日權重：週末較高，平日較低（可以根據需要調整）
    weights = np.array([1.2 if date.weekday() >= 5 else 1.0 for date in days_in_month])
    weights /= weights.sum()  # 將權重標準化，使總和為1
    
    # 計算每日數據
    daily_quantity = (total_quantity * weights).round(2)
    
    # 返回 DataFrame
    return pd.DataFrame({
        'date': days_in_month,
        f'{column}_daily': daily_quantity
    })

# 對所有列應用展開邏輯
daily_data = pd.DataFrame({'date': []})  # 用於存放展開數據
for column in ["Brent", "Dubai", "Unknown", "WTI"]:
    expanded_data = pd.concat([expand_to_daily(row, column) for _, row in df.iterrows()])
    daily_data = pd.merge(daily_data, expanded_data, on='date', how='outer') if not daily_data.empty else expanded_data

# 打印結果
print(daily_data)
daily_data.to_csv("daily_export_data.csv", index=False, encoding='utf-8')

           date  Brent_daily  Dubai_daily  Unknown_daily  WTI_daily
0    2020-01-01      1392.70       860.18        5371.32   35048.31
1    2020-01-02      1392.70       860.18        5371.32   35048.31
2    2020-01-03      1392.70       860.18        5371.32   35048.31
3    2020-01-04      1671.24      1032.22        6445.58   42057.98
4    2020-01-05      1671.24      1032.22        6445.58   42057.98
...         ...          ...          ...            ...        ...
1730 2024-09-26      2744.97       321.60        3886.32   35675.13
1731 2024-09-27      2744.97       321.60        3886.32   35675.13
1732 2024-09-28      3293.96       385.92        4663.58   42810.15
1733 2024-09-29      3293.96       385.92        4663.58   42810.15
1734 2024-09-30      2744.97       321.60        3886.32   35675.13

[1735 rows x 5 columns]


針對price進行插值填補，修補缺失值

In [14]:
file_path = "data/price.csv"
df = pd.read_csv(file_path)

print(df.head())  # 顯示前五筆資料

       date    WTI  Dubai  Brent  exchange rate
0  2020/1/2  61.14  65.41  67.43         30.114
1  2020/1/3  62.97  67.95  69.72         30.105
2  2020/1/6  63.24  68.16  69.82         30.110
3  2020/1/7  62.67  69.35  70.51         30.082
4  2020/1/8  59.62  65.44  66.57         30.087


In [18]:
# 將日期轉換為 datetime 格式
df['date'] = pd.to_datetime(df['date'])

# 1. 生成完整日期範圍
full_date_range = pd.date_range(start="2020-01-01", end=df['date'].max(), freq='D')

# 2. 補齊缺失日期，並以 NaN 填充缺失數據
df = df.set_index('date')  # 將日期設為索引
df = df.reindex(full_date_range, fill_value=None)  # 填補缺失日期
df.index.name = 'date'  # 重命名索引列為 'date'

# 3. 使用插值填補缺失值
df = df.interpolate(method='linear', limit_direction='both')  # 線性插值

df = df.round(2)

# 4. 重置索引為普通列
df = df.reset_index()


# 打印結果
print(df)

df.to_csv("daily_price.csv", index=False, encoding='utf-8')

           date    WTI  Dubai  Brent  exchange rate
0    2020-01-01  61.14  65.41  67.43          30.11
1    2020-01-02  61.14  65.41  67.43          30.11
2    2020-01-03  62.97  67.95  69.72          30.10
3    2020-01-04  63.06  68.02  69.75          30.11
4    2020-01-05  63.15  68.09  69.79          30.11
...         ...    ...    ...    ...            ...
1804 2024-12-09  68.65  72.55  73.73          32.47
1805 2024-12-10  68.85  72.69  73.64          32.44
1806 2024-12-11  70.57  74.35  74.38          32.51
1807 2024-12-12  70.25  74.22  73.52          32.49
1808 2024-12-13  71.54  75.16  74.89          32.51

[1809 rows x 5 columns]


計算歷史每日漲幅

In [3]:
file_path = "all_data.csv"
df = pd.read_csv(file_path)

print(df.head())  # 顯示前五筆資料

       date  WTI_export  Dubai_export  Brent_export  Unknown_export  \
0  2020/1/1    35048.31        860.18       1392.70         5371.32   
1  2020/1/2    35048.31        860.18       1392.70         5371.32   
2  2020/1/3    35048.31        860.18       1392.70         5371.32   
3  2020/1/4    42057.98       1032.22       1671.24         6445.58   
4  2020/1/5    42057.98       1032.22       1671.24         6445.58   

   WTI_price  Dubai_price  Brent_price  exchange rate  
0      61.14        65.41        67.43          30.11  
1      61.14        65.41        67.43          30.11  
2      62.97        67.95        69.72          30.10  
3      63.06        68.02        69.75          30.11  
4      63.15        68.09        69.79          30.11  


In [10]:
# 計算每日漲幅 (百分比變化)
df['WTI_change'] = df['WTI_price'].pct_change() * 100  # 計算 WTI 的漲幅
df['Dubai_change'] = df['Dubai_price'].pct_change() * 100  # 計算 Dubai 的漲幅
df['Brent_change'] = df['Brent_price'].pct_change() * 100  # 計算 Brent 的漲幅

# 將小數點保留到第二位
df = df.round(2)

# 打印結果
print(df)
df.to_csv("data.csv", index=False, encoding='utf-8')

            date  WTI_export  Dubai_export  Brent_export  Unknown_export  \
0       2020/1/1    35048.31        860.18       1392.70         5371.32   
1       2020/1/2    35048.31        860.18       1392.70         5371.32   
2       2020/1/3    35048.31        860.18       1392.70         5371.32   
3       2020/1/4    42057.98       1032.22       1671.24         6445.58   
4       2020/1/5    42057.98       1032.22       1671.24         6445.58   
...          ...         ...           ...           ...             ...   
1804   2024/12/9         NaN           NaN           NaN             NaN   
1805  2024/12/10         NaN           NaN           NaN             NaN   
1806  2024/12/11         NaN           NaN           NaN             NaN   
1807  2024/12/12         NaN           NaN           NaN             NaN   
1808  2024/12/13         NaN           NaN           NaN             NaN   

      WTI_price  Dubai_price  Brent_price  exchange rate  WTI_change  \
0         61.14

運用線性插植處理美金指標的缺失值

In [37]:
file_path = "data/US Dollar Index Historical Data (1).csv"
df = pd.read_csv(file_path)

print(df.head())  # 顯示前五筆資料

         Date  Price
0  01/01/2020  96.39
1  01/02/2020  96.85
2  01/03/2020  96.84
3  01/06/2020  96.67
4  01/07/2020  97.00


In [33]:
print(df.dtypes)


Date      object
Price    float64
dtype: object


In [40]:
# 1. 確保日期格式正確
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')  # 根據數據格式轉換

# 2. 設置日期為索引
df = df.set_index('Date')

# 3. 生成完整的日期範圍
full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# 4. 補全缺失日期
df = df.reindex(full_date_range, fill_value=None)
df.index.name = 'Date'

# 5. 對 `Price` 列進行線性插值
df['Price'] = df['Price'].interpolate(method='linear')

df = df.round(2)

# 6. 重置索引為普通列
df = df.reset_index()

# 7. 檢查結果
print(df.head())
df.to_csv("data/daily_US Dollar Index Historical Data.csv", index=False, encoding='utf-8')

        Date  Price
0 2020-01-01  96.39
1 2020-01-02  96.85
2 2020-01-03  96.84
3 2020-01-04  96.78
4 2020-01-05  96.73
